In [ ]:
import glob, os
import zipfile
from read_roi import read_roi_zip 
import SimpleITK as sitk
import numpy as np
from collections import OrderedDict

from  scipy import ndimage

import matplotlib.pyplot as plt

from PIL import Image, ImageDraw 
from PIL import ImagePath
import cv2

import pandas as pd
def win_scale(data, wl, ww, dtype, out_range):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    
    data_new = np.empty(data.shape, dtype=np.double)
    data_new.fill(out_range[1]-1)
    
    data_new[data <= (wl-ww/2.0)] = out_range[0]
    
    data_new[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))] = ((data[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))]-(wl-0.5))/(ww-1.0)+0.5)*(out_range[1]-out_range[0])+out_range[0]
    
    data_new[data > (wl+ww/2.0)] = out_range[1]-1
    
    return data_new.astype(dtype)

In [ ]:
roiPath = '../segment/'
dcmList = sorted(glob.glob('../test_dcm/*'))
p_num= []

for pn in range(len(dcmList)):
    p_num.append(dcmList[pn][dcmList[pn].rindex('/')+1:])

In [ ]:
#########check_file, excel type

xlsx = pd.read_excel('../../data/dvtList_20210120.xlsx', engine='openpyxl')
dcmList = sorted(glob.glob('../../data/dvt/*'))
dcms=sorted(glob.glob(dcmList[0]+'/*.dcm'))
f_numpath = dcmList
f_numStr=[]
f_numInt=[]
f_date=[]



# for pn in range(len(p_numpath)):
fn=0
f_numStr.append(f_numpath[fn].split('\\')[-1])
f_numInt.append(int(f_numpath[fn].split('\\')[-1]))
f_dcm= sitk.ReadImage(dcms[0])
f_date.append(f_dcm.GetMetaData('0008|0020'))

print('p_numstr:type={}, ex={}'.format(f_numStr[0], f_numStr[0]))
print('p_numInt:type={}, ex={}'.format(f_numInt[0], f_numInt[0]))
print('p_date:type={}, ex={}'.format(f_date[0], f_date[0]))

In [ ]:
xlsx = pd.read_excel('../../local/data/dvtList_20210120.xlsx', engine='openpyxl',)
dcmList = sorted(glob.glob('../../local/data/dvt/*'))
p_numpath = dcmList
p_numStr=[]
p_numInt=[]
for pn in range(len(p_numpath)):
    p_numStr.append(p_numpath[pn][p_numpath[pn].rindex('/')+1:])
    p_numInt.append(int(p_numpath[pn][p_numpath[pn].rindex('/')+1:]))
print(p_numStr[0])
print(p_numInt[0])
print(len(p_numpath),len(p_numStr),len(p_numInt))

p_ids = []
ww_list = []
wl_list = []
for ids in range(len(xlsx)):
    p_ids.append(xlsx['ID'][ids])
    ww_list.append(xlsx['WW'][ids])
    wl_list.append(xlsx['WL'][ids])
print(len(p_ids), len(ww_list), len(wl_list))    

print(p_ids.index(int(p_numStr[0])))
print(p_ids[p_ids.index(p_numInt[0])], p_numStr[0])
ww=[]
wl=[]
for tt in range(len(p_numStr)):
    if p_numInt[tt] not in p_ids:
        print(p_numStr[tt], 'files not in list')
        ww.append(ww_list[p_ids.index(p_numInt[tt][:8])])
        wl.append(wl_list[p_ids.index(p_numInt[tt][:8])])
        
    else:
        print('number:', tt, 'match:', p_ids[p_ids.index(p_numInt[tt])], p_numStr[tt])
        if p_ids[p_ids.index(p_numInt[tt])] != p_numStr[tt]:
            print('No matching')
        else:
            ww.append(ww_list[p_ids.index(p_numStr[tt])])
            wl.append(wl_list[p_ids.index(p_numStr[tt])])

In [ ]:
for whl in range(len(p_num)):
# for whl in range(2):
    
    abroiPath= '../segment/'+ p_num[whl]+'/abnormal.zip'
    nroiPath='../segment/'+ p_num[whl]+'/normal.zip'

    abr = read_roi_zip(abroiPath)
    nr =  read_roi_zip(nroiPath)

    abn_odKey = list(OrderedDict(abr).keys())
    n_odKey = list(OrderedDict(nr).keys())

    abArea=[]
    nArea=[]
    ab_x = []
    ab_y = []
    n_x = []
    n_y = []
    
    
       
    for ak in range(len(abn_odKey)):
        
        if len(str(int(abn_odKey[ak].split('-')[0])-1))==2:
            match_num = '00'+str(int(abn_odKey[ak].split('-')[0])-1)

        elif len(str(int(abn_odKey[ak].split('-')[0])-1))==3:
            match_num = '0'+str(int(abn_odKey[ak].split('-')[0])-1)
        else:
            print(str(int(abn_odKey[ak].split('-')[0])-1))
        
        ab_x_min = min(abr[abn_odKey[ak]]['x'])
        ab_y_min = min(abr[abn_odKey[ak]]['y'])
        ab_x_max = max(abr[abn_odKey[ak]]['x'])
        ab_y_max = max(abr[abn_odKey[ak]]['y'])
# 
#         dcm_dcm = sitk.ReadImage(dcmList[whl]+'/'+p_num[whl]+'_'+'0'+str(int(abn_odKey[ak].split('-')[0])-1)+'.dcm')
        dcm_dcm = sitk.ReadImage(dcmList[whl]+'/'+p_num[whl]+'_'+match_num+'.dcm')
        dcm_img = sitk.GetArrayFromImage(dcm_dcm)[0]
        img_array = win_scale(dcm_img, wl[whl], ww[whl], np.uint8, (0, 255))

    
        bbb = np.zeros((ab_y_max-ab_y_min, ab_x_max-ab_x_min,1), dtype= np.uint8)
        bbb[:,:,0] = img_array[ab_y_min:ab_y_max, ab_x_min:ab_x_max]
        
        abArea.append((ab_y_max-ab_y_min)*(ab_x_max-ab_x_min))
        ab_y.append(ab_y_max-ab_y_min)
        ab_x.append(ab_x_max-ab_x_min)
#         bbb = np.zeros((ab_x_max-ab_x_min, ab_y_max-ab_y_min, 1), dtype= np.uint8)
#         bbb[:,:,0] = img_array[ab_x_min:ab_x_max,ab_y_min:ab_y_max]
        
#     for nk in range(len(n_odKey)):
        
        n_x_min = min(nr[n_odKey[ak]]['x'])
        n_y_min = min(nr[n_odKey[ak]]['y'])
        n_x_max = max(nr[n_odKey[ak]]['x'])
        n_y_max = max(nr[n_odKey[ak]]['y'])

#         dcm_dcm = sitk.ReadImage(dcmList[whl]+'/'+p_num[whl]+'_'+abn_odKey[ak].split('-')[0]+'.dcm')
#         dcm_img = sitk.GetArrayFromImage(dcm_dcm)[0]
#         img_array = win_scale(dcm_img, 80, 380, np.uint8, (0, 255))

        nnn = np.zeros((n_y_max-n_y_min, n_x_max-n_x_min,1), dtype= np.uint8)
        nnn[:,:,0] = img_array[n_y_min:n_y_max, n_x_min:n_x_max]
        
        nArea.append((n_y_max-n_y_min)*(n_x_max-n_x_min))
        n_y.append(n_y_max-n_y_min)
        n_x.append(n_x_max-n_x_min)

In [ ]:
size = 32
ab_img = sorted(glob.glob('../test_img_origin/abnormal/*.png'))
n_img = sorted(glob.glob('../test_img_origin/normal/*.png'))
ab_np = np.zeros((51,32,32,1), dtype=np.uint8)
n_np = np.zeros((51,32,32,1), dtype= np.uint8)
i = 0
for ab_whl in range(len(ab_img)):
    
    ab_im = Image.open(ab_img[ab_whl])
    n_im = Image.open(n_img[ab_whl])
    aw ,ah = ab_im.size
    nw ,nh = n_im.size
    ab_scale = size/max(aw, ah)
    n_scale = size/max(nw, nh)
    if aw > ah:
        resized1 = ab_im.resize((size, round(ah*ab_scale)), resample=Image.BILINEAR, box=None)
#         resized2 = im.resize((size, round(h*scale)), resample=Image.BILINEAR, box=None)
        
    else:
        resized1 = ab_im.resize((round(aw*ab_scale), size), resample=Image.BILINEAR, box=None)
#         resized2 = im.resize((round(w*scale), size), resample=Image.BILINEAR, box=None)
    if nw > nh:
        resized2 = n_im.resize((size, round(nh*n_scale)), resample=Image.BILINEAR, box=None)
#         resized2 = im.resize((size, round(h*scale)), resample=Image.BILINEAR, box=None)
        
    else:
        resized2 = n_im.resize((round(nw*n_scale), size), resample=Image.BILINEAR, box=None)   
        
        
    ab_array = np.array(resized1)
    n_array = np.array(resized2)
    
    ab_np[i][:resized1.size[1], :resized1.size[0],0] = ab_array
    n_np[i][:resized2.size[1], :resized2.size[0],0] = n_array
    
    i+=1
    
np.save('../test_img/32/abnormal_32_bilinear_1', ab_np)
np.save('../test_img/32/normal_32_bilinear_1', n_np)

In [ ]:
train_img = np.ndarray((70, 32,32, 1), dtype=np.uint8)
test_img = np.ndarray((32,32, 32,1), dtype=np.uint8)
train_label = []
test_label= []
for lab in range(102):
    if lab< 35:
        train_label.append(0)
        train_img[lab][:,:,0] = ab_np[lab][:,:,0]
    elif lab >= 35 and lab<70:
        train_label.append(1)
        train_img[lab][:,:,0] = n_np[lab-35][:,:,0]
    elif lab >= 70 and lab <86:
        test_label.append(0)
        test_img[lab-70][:,:,0] = ab_np[lab-35][:,:,0]
    else:
        test_label.append(1)
        test_img[lab-70][:,:,0] = n_np[lab-51][:,:,0]

In [ ]:
train_label=np.array(train_label)
test_label = np.array(test_label)
np.save('../train_img_32_bilinear', train_img)
np.save('../test_img_32_bilinear', test_img)
np.save('../test_label_32_bilinear', test_label)
np.save('../train_label_32_bilinear', train_label)